In [55]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import empyrical
import datetime
import numpy as np
import pandas as pd
from backtrader.utils.py3 import iteritems
import pdb
import pyfolio as pf
import decimal
import pathlib

import matplotlib

In [84]:
# get data
def get_data_feed_by_name(data_feed_name, start, end):
    return bt.feeds.YahooFinanceData(dataname=data_feed_name,period='d', fromdate=start,
                                todate=end)

def get_local_data_feed_by_name(data_filename,start,end):
    return bt.feeds.YahooFinanceCSVData(
        dataname=pathlib.Path.cwd().parent.joinpath('data','VUSA.DE.csv').as_posix(),
        # Do not pass values before this date
        fromdate=start,
        # Do not pass values after this date
        todate=end,
        reverse=False)

start=datetime.datetime(2020,1,1)
end=datetime.datetime(2020,12,1)
data_vusa_csv = get_local_data_feed_by_name('VUSA.DE.csv',start,end)
data_stoxx_csv = get_local_data_feed_by_name('EXSA.DE.csv',start,end)

data_vusa = get_data_feed_by_name('VUSA.DE',start,end)
data_stoxx = get_data_feed_by_name('EXSA.DE',start,end)

In [85]:
etf_allocation = {'vusa':(data_vusa,0.5),
                  'stoxx':(data_stoxx,0.5),
                  #'emerging_markets':(data_emerging_markets,0.333),
                 #'nasdaq100':(data_nasdaq100,0.333),
            #'china':(data_china,0.05),
                 #'pacific':(data_pacific,0.05),'bonds':(data_bonds,0.1),
                 #'japan':(data_japan,0.03), 'tech':(data_tech,0.02)
                }

etf_allocation_csv = {'vusa':(data_vusa_csv,0.8),
                      'stoxx':(data_stoxx_csv,0.2)
                 #'nasdaq100':(data_nasdaq100_csv,0.25),'china':(data_china_csv,0.05),
                 #'pacific':(data_pacific_csv,0.05),
                      #'pacific':(data_pacific,0.05),
                 #     'bonds':(data_bonds_csv,0.05),
                 #'japan':(data_japan_csv,0.05), 'tech':(data_tech_csv,0.05)
                     }
                 
print (sum([v[1] for k,v in etf_allocation.items()]))

1.0


In [86]:
# define strategy
class BuyAndHoldTarget(bt.Strategy):
    # ToDo - Add etf allocation to init method
    params = dict(
        monthly_cash=700.0,  # amount of cash to buy every month
    )

    def start(self):
        # Activate the fund mode and set the default value at 100
        self.broker.set_fundmode(fundmode=True, fundstartval=100.00)
        self.cash_start = self.broker.get_cash()
        self.val_start = 100.0

        # Add a timer which will be called on the 1st trading day of the month
        self.add_timer(
            bt.timer.SESSION_END,  # when it will be called
            monthdays=[1],  # called on the 1st day of the month
            monthcarry=True,  # called on the 2nd day if the 1st is holiday
        )

    def buy_fixed_cash_amount_dummy(self, target_value, data_name):

        data_feed = self.getdatabyname(name=data_name)
        #return self.buy(exectype=bt.Order.Market,size=size,data=data_feed)
        value = self.broker.getvalue(datas=[data_feed])
        comminfo = self.broker.getcommissioninfo(data_feed)
        price = data_feed.close[0]     
        size = self.getsize(comminfo,price, target_value)
        print ('size : {}, price: {}, total_order amount: {}'.format(size,price,size*price))
        return self.buy(data=data_feed, size=size, price=price)

    def getsize(self, comminfo, price, cash):
        if not comminfo._stocklike:
            return np.float(comminfo.p.leverage * (cash / comminfo.get_margin(price)))
        return np.float(1 * (cash / price))
        
    def notify_timer(self, timer, when, *args, **kwargs):
        # Add the influx of monthly cash to the broker
        self.broker.add_cash(self.p.monthly_cash)
        
        # buy available cash - we neglect the remaining value of cash, only invest the monthly amount
        #target_value = self.broker.getcash() + self.p.monthly_cash
        target_value = self.p.monthly_cash
        order_list = []
        
        for data_name,(data_feed,percent_allocation) in etf_allocation_csv.items():
            target = target_value*percent_allocation
            print ('buying {} from {} when {}, total target {}'.format(target, data_name, when, target_value))
            order = self.buy_fixed_cash_amount_dummy(target,data_name)
            if not order:
                print ('could not buy from {} when {}'.format(data_name, when))

    def stop(self):
        self.roi = (self.broker.get_value() / self.cash_start) - 1.0
        self.froi = self.broker.get_fundvalue() - self.val_start
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))
        print('Fund Value: {:.2f}%'.format(self.froi))
        print ('broker fund value {} broker val start {}'.format(self.broker.get_fundvalue(),self.val_start))

In [87]:
# run backtest
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(BuyAndHoldTarget)
cerebro.broker.set_cash(0.000001)
cerebro.broker.set_fundmode(True)


#for data_name,(data_feed,percent_allocation) in etf_allocation.items():    
#    cerebro.adddata(data_feed, name=data_name)
for data_name,(data_feed,percent_allocation) in etf_allocation_csv.items():
    cerebro.adddata(data_feed, name=data_name)


#cerebro.adddata(data_vusa_csv, name='vusa')
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Observers
cerebro.addobserver(bt.observers.TimeReturn, timeframe=bt.TimeFrame.Days)

# Analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio', timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.PositionsValue, headers=True, cash=True, _name='mypositions')
cerebro.addanalyzer(bt.analyzers.TimeReturn,_name='timereturn')

results = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 0.00
buying 560.0 from vusa when 2020-01-02 23:59:59.999990, total target 700.0
size : 10.19850664724094, price: 54.91, total_order amount: 560.0
buying 140.0 from stoxx when 2020-01-02 23:59:59.999990, total target 700.0
size : 2.549626661810235, price: 54.91, total_order amount: 140.0
buying 560.0 from vusa when 2020-02-03 23:59:59.999990, total target 700.0
size : 10.026857654431513, price: 55.85, total_order amount: 560.0
buying 140.0 from stoxx when 2020-02-03 23:59:59.999990, total target 700.0
size : 2.5067144136078783, price: 55.85, total_order amount: 140.0
buying 560.0 from vusa when 2020-03-03 23:59:59.999990, total target 700.0
size : 10.792060127192137, price: 51.89, total_order amount: 560.0
buying 140.0 from stoxx when 2020-03-03 23:59:59.999990, total target 700.0
size : 2.6980150317980343, price: 51.89, total_order amount: 140.0
buying 560.0 from vusa when 2020-04-01 23:59:59.999990, total target 700.0
size : 12.897282358360203, price: 43.42, 

In [88]:
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio',)
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items() #<--positions only holds data for one symbol here

p = strat.analyzers.getbyname('mypositions').get_analysis()
mypositions = [[k] + v for k, v in iteritems(p)] #<-- Now positions will hold data for all symbols
cols = mypositions.pop(0)  # headers are in the first entry
mypositions = pd.DataFrame.from_records(mypositions, index=cols[0], columns=cols)
mypositions.index = pd.DatetimeIndex(mypositions.index) 

In [89]:
transactions.head(10)

,amount,price,sid,symbol,value
date,,,,,
2020-01-03 23:59:59.999989+00:00,10.198507,54.91,0,vusa,-560.000000
2020-01-03 23:59:59.999989+00:00,2.549627,54.91,1,stoxx,-140.000000
2020-02-04 23:59:59.999989+00:00,10.026858,56.26,0,vusa,-564.111012
2020-03-04 23:59:59.999989+00:00,10.792060,51.82,0,vusa,-559.244556
2020-03-04 23:59:59.999989+00:00,2.698015,51.82,1,stoxx,-139.811139
2020-04-02 23:59:59.999989+00:00,12.897282,43.38,0,vusa,-559.484109
2020-04-02 23:59:59.999989+00:00,3.224321,43.38,1,stoxx,-139.871027
2020-05-05 23:59:59.999989+00:00,11.426240,49.77,0,vusa,-568.683942
2020-05-05 23:59:59.999989+00:00,2.856560,49.77,1,stoxx,-142.170986


In [90]:
mypositions.head(20)

,vusa,stoxx,cash
Datetime,,,
2020-01-02,0.000000,0.000000,7.000000e+02
2020-01-03,560.815881,140.203970,1.000000e-06
2020-01-06,559.592060,139.898015,1.000000e-06
2020-01-07,563.365507,140.841377,1.000000e-06
2020-01-08,566.119104,141.529776,1.000000e-06
2020-01-09,570.912402,142.728101,1.000000e-06
2020-01-10,570.504462,142.626115,1.000000e-06
2020-01-13,569.688581,142.422145,1.000000e-06
2020-01-14,572.034238,143.008559,1.000000e-06
